In [1]:
from sklearn.model_selection import train_test_split
import pickle
import os
import h5py
from PIL import Image
import numpy as np
import tf_transformations as tf_trans
import numpy as np
import math
import copy

In [3]:


# Load the dataset

def load_dataset(output_dir):
    # Define paths to your pickle files
    image_file = os.path.join(output_dir, 'images.h5')
    laser_file = os.path.join(output_dir, 'lasers.pkl')
    odom_file = os.path.join(output_dir, 'odoms.pkl')
    goal_odom_file = os.path.join(output_dir, 'goal_odoms.pkl')
    velocity_file = os.path.join(output_dir, 'velocities.pkl')
    tf_file = os.path.join(output_dir, 'tfs.pkl')
    
    # Load the pickle files
    with open(laser_file, 'rb') as f:
        lasers = pickle.load(f)
    with open(odom_file, 'rb') as f:
        current_poses = pickle.load(f)
    with open(goal_odom_file, 'rb') as f:
        goal_poses = pickle.load(f)
    with open(velocity_file, 'rb') as f:
        velocities = pickle.load(f)
    with open(tf_file, 'rb') as f:
        tfs = pickle.load(f)
    # print('loading images')
    # with h5py.File(image_file, 'r') as f:
    #     image_keys = list(f.keys())
    #     images = [Image.fromarray(f[key][:]) for key in image_keys]
    # print('loaded images')
    # Split velocities into linear and angular components
    linear_velocities = np.array(velocities)[:, :2]  # Linear velocity (x, y)
    angular_velocities = np.array(velocities)[:, 2]  # Angular velocity (z)
    
    return  list(zip(lasers, current_poses, goal_poses, linear_velocities, angular_velocities, tfs))


output_dir = "/home/nigitha/ros2_ws_rnd/src/dataset/corr1"  # Replace with your output directory path
data = load_dataset(output_dir)
print('loaded data')

loaded data


In [4]:
import transforms3d

# Function to invert a transformation (translation + rotation)
def invert_transform(translation, rotation):
    # Convert the quaternion to a rotation matrix (3x3)
    rotation_matrix = transforms3d.quaternions.quat2mat([rotation[3], rotation[0], rotation[1], rotation[2]])[:3, :3]
    # Invert the rotation matrix (transpose of a rotation matrix is its inverse)
    rotation_matrix_inv = np.transpose(rotation_matrix)
    
    # Invert the translation (apply the inverse rotation to the negative translation)
    translation_inv = -np.dot(rotation_matrix_inv, [translation[0], translation[1], translation[2]])
    
    # Create the inverse quaternion (negate the vector part, keep the scalar part the same)
    rotation_inv = transforms3d.quaternions.qinverse([rotation[3], rotation[0], rotation[1], rotation[2]])
    return translation_inv, rotation_inv

# Function to apply the transformation (translation + rotation) to a point
def transform_point(translation, rotation, point):
    # Convert the point to a homogeneous vector (x, y, z, 1)
    point_homogeneous = np.array([point[0], point[1], point[2], 1.0])
    
    # Create the translation matrix (4x4)
    translation_matrix = np.identity(4)
    translation_matrix[0, 3] = translation[0]
    translation_matrix[1, 3] = translation[1]
    translation_matrix[2, 3] = translation[2]
    # Create the transformation matrix from translation and rotation
    rotation_matrix = np.identity(4)
    rotation_matrix[:3, :3] = transforms3d.quaternions.quat2mat(rotation)[:3, :3]
    # Combine translation and rotation into a single transformation matrix
    transformation_matrix = np.dot(translation_matrix, rotation_matrix)
    # Apply the transformation to the point
    transformed_point = np.dot(transformation_matrix, point_homogeneous)
    
    
    # Return the transformed point (x, y, z)
    return transformed_point[:3]

# Function to transform a point from odom to base_link frame
def transform_pose_to_base_link(translation, rotation, point_in_odom):
    # Get the inverse of the transformation
    translation_inv, rotation_inv = invert_transform(translation, rotation)
    
    # Convert the point from odom to base_link using the inverted transformation
    point_in_base_link = transform_point(translation_inv, rotation_inv, point_in_odom)
    
    return point_in_base_link

In [21]:
lasers = []
current_poses = []
goal_poses = []
linear_velocities = []
angular_velocities = []
tfs = []
goal_distances = []
goal_angles = []
target_distance = 0.2

for i, (_, _, goal_pose1, _, _, _) in enumerate(data):
    for j, (laser2, current_pose2, _, linear_velocity2, angular_velocity2, tf2) in enumerate(data[i:]):          
        distance = np.sqrt((goal_pose1[0] - current_pose2[0]) ** 2 + (goal_pose1[1] - current_pose2[1]) ** 2)
        if distance >= target_distance:
            goal_baselink = transform_pose_to_base_link(tf2[0], tf2[1], goal_pose1)
            goal_distances.append(distance)
            goal_angles.append(math.atan2(goal_baselink[1],goal_baselink[0]))
            lasers.append(laser2)
            current_poses.append(current_pose2)
            goal_poses.append(goal_pose1)
            linear_velocities.append(linear_velocity2)
            angular_velocities.append(angular_velocity2)
            tfs.append(tf2)
        else:
            break

In [35]:
zipped = list(zip(current_poses, goal_poses, goal_angles, goal_distances, linear_velocities, angular_velocities))
for i, (current_pose, goal_pose, goal_angle, goal_distance, linear_velocity, angular_velocity) in enumerate(zipped):
    if goal_angle>0.3:
        print(i)
        break

45314


In [36]:
# zipped = list(zip(current_poses, goal_poses, goal_angles, goal_distances, linear_velocities, angular_velocities))
for i, (current_pose, goal_pose, goal_angle, goal_distance, linear_velocity, angular_velocity) in enumerate(zipped):
    if i >45300 and i<46000:
        print(current_pose, '\n' , goal_pose, '\n' ,  goal_angle,  '\n' , 
              goal_distance,  '\n' , linear_velocity,  '\n' , angular_velocity,  '\n' )
        
    if i> 46000:
        break

(5.563766355365851, 7.330616965685829, 0.5070504874440165) 
 (5.739912825072267, 8.110441630106374, 0.7479882492278541) 
 0.2851262594251547 
 0.7994711289525399 
 [ 9.30498384e-03 -1.56024052e-05] 
 0.0006845371674857308 

(5.56612037135087, 7.334839582865533, 0.5070192767965231) 
 (5.739912825072267, 8.110441630106374, 0.7479882492278541) 
 0.2869170301625646 
 0.7948347958253257 
 [ 8.38824954e-03 -1.32714985e-05] 
 -0.00033126081518392936 

(5.568644680926994, 7.339356387977423, 0.5070361548333853) 
 (5.739912825072267, 8.110441630106374, 0.7479882492278541) 
 0.28874603109708197 
 0.789876716853984 
 [ 8.95484569e-03 -2.33106492e-05] 
 -0.0001374769104435722 

(5.571000260651034, 7.3435695024291565, 0.5070501600257985) 
 (5.739912825072267, 8.110441630106374, 0.7479882492278541) 
 0.2904785859742499 
 0.785254299337189 
 [ 7.09141812e-03 -1.77617966e-05] 
 0.00011571519768240968 

(5.573397690160698, 7.34786180546914, 0.5071979410260312) 
 (5.739912825072267, 8.110441630106374, 0.

In [29]:

import tf_transformations as tf_trans
import numpy as np

# Function to invert a transformation (translation + rotation)
def invert_transform(translation, rotation):
    # Convert the quaternion to a rotation matrix (3x3)
    rotation_matrix = tf_trans.quaternion_matrix([rotation.x, rotation.y, rotation.z, rotation.w])[:3, :3]
    # Invert the rotation matrix (transpose of a rotation matrix is its inverse)
    rotation_matrix_inv = np.transpose(rotation_matrix)
    
    # Invert the translation (apply the inverse rotation to the negative translation)
    translation_inv = -np.dot(rotation_matrix_inv, [translation.x, translation.y, translation.z])
    
    # Create the inverse quaternion (negate the vector part, keep the scalar part the same)
    rotation_inv = tf_trans.quaternion_inverse([rotation.x, rotation.y, rotation.z, rotation.w])
    return translation_inv, rotation_inv

# Function to apply the transformation (translation + rotation) to a point
def transform_point(translation, rotation, point):
    # Convert the point to a homogeneous vector (x, y, z, 1)
    point_homogeneous = np.array([point[0], point[1], 0.0, 1.0])
    
    # Create the translation matrix (4x4)
    translation_matrix = np.identity(4)
    translation_matrix[0, 3] = translation[0]
    translation_matrix[1, 3] = translation[1]
    translation_matrix[2, 3] = translation[2]
    print(rotation)
    print('rotation')
    # Create the rotation matrix from the quaternion (4x4)
    rotation_matrix = np.identity(4)
    rotation_matrix[:3, :3] = tf_trans.quaternion_matrix(rotation)[:3, :3]
    print('rotation_matrix')
    print(rotation_matrix)
    # Combine translation and rotation into a single transformation matrix
    transformation_matrix = np.dot(translation_matrix, rotation_matrix)
    print('transformation_matrix')
    print(transformation_matrix)
    # Apply the transformation to the point
    transformed_point = np.dot(transformation_matrix, point_homogeneous)
    
    # Return the transformed point (x, y, z)
    return transformed_point[:3]

def transform_pose_to_base_link(translation, rotation, point_in_odom):

    translation_inv, rotation_inv = invert_transform(translation, rotation)


    # Convert point from odom to base_link using the inverted transform
    point_in_base_link = transform_point(translation_inv, rotation_inv, point_in_odom)
    
    return point_in_base_link

for i in range (200):
    if linear_velocities[i][0]>0.001:
        print(i)
        print(current_poses[i], '\n', goal_poses[i], linear_velocities[i], angular_velocities[i], tfs[i])
        goal_base = transform_pose_to_base_link(tfs[i][0], tfs[i][1], goal_poses[i])
        print(goal_base)
        print(math.atan2(goal_base[1],goal_base[0]))
        theta = math.atan2(linear_velocities[i][1], linear_velocities[i][0])
    
        # Update the direction based on angular velocity
        theta_new = theta + angular_velocities[i] 
    
        # Normalize theta_new to the range [-pi, pi]
        theta_new = math.atan2(math.sin(theta_new), math.cos(theta_new))
        print(theta)
        print()
        break




18
(0.8304349815579143, 4.399584956767328, -0.7982134099140811) 
 (0.6456304800790127, 3.418200698165892, -0.7527153934999217) [4.46613975e-03 9.55571903e-05] -0.0010603089773211824 (geometry_msgs.msg.Vector3(x=0.8304349815579143, y=4.399584956767328, z=0.0), geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=-0.7982134099140811, w=0.6023747606211064), 1724064038266487612)
[-0.         -0.          0.79821341  0.60237476]
rotation
rotation_matrix
[[-0.2742893  -0.96164722  0.          0.        ]
 [ 0.96164722 -0.2742893   0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
transformation_matrix
[[-0.2742893  -0.96164722  0.          4.45862808]
 [ 0.96164722 -0.2742893   0.          0.40817356]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
[0.99443534 0.09146646 0.        ]
0.09172021851277114
0.021392660799079095



In [20]:
import numpy as np
import transforms3d

import transforms3d

# Function to invert a transformation (translation + rotation)
def invert_transform(translation, rotation):
    # Convert the quaternion to a rotation matrix (3x3)
    rotation_matrix = transforms3d.quaternions.quat2mat([rotation[3], rotation[0], rotation[1], rotation[2]])[:3, :3]
    # Invert the rotation matrix (transpose of a rotation matrix is its inverse)
    rotation_matrix_inv = np.transpose(rotation_matrix)
    
    # Invert the translation (apply the inverse rotation to the negative translation)
    translation_inv = -np.dot(rotation_matrix_inv, [translation[0], translation[1], translation[2]])
    
    # Create the inverse quaternion (negate the vector part, keep the scalar part the same)
    rotation_inv = transforms3d.quaternions.qinverse([rotation[3], rotation[0], rotation[1], rotation[2]])
    return translation_inv, rotation_inv

# Function to apply the transformation (translation + rotation) to a point
def transform_point(translation, rotation, point):
    # Convert the point to a homogeneous vector (x, y, z, 1)
    point_homogeneous = np.array([point[0], point[1], point[2], 1.0])
    
    # Create the translation matrix (4x4)
    translation_matrix = np.identity(4)
    translation_matrix[0, 3] = translation[0]
    translation_matrix[1, 3] = translation[1]
    translation_matrix[2, 3] = translation[2]
    # Create the transformation matrix from translation and rotation
    rotation_matrix = np.identity(4)
    rotation_matrix[:3, :3] = transforms3d.quaternions.quat2mat(rotation)[:3, :3]
    # Combine translation and rotation into a single transformation matrix
    transformation_matrix = np.dot(translation_matrix, rotation_matrix)
    # Apply the transformation to the point
    transformed_point = np.dot(transformation_matrix, point_homogeneous)
    
    
    # Return the transformed point (x, y, z)
    return transformed_point[:3]

# Function to transform a point from odom to base_link frame
def transform_pose_to_base_link(translation, rotation, point_in_odom):
    # Get the inverse of the transformation
    translation_inv, rotation_inv = invert_transform(translation, rotation)
    
    # Convert the point from odom to base_link using the inverted transformation
    point_in_base_link = transform_point(translation_inv, rotation_inv, point_in_odom)
    
    return point_in_base_link

for i, (_, current_pose, goal_pose, linear_velocity, angular_velocity, tf) in enumerate(data[i:]):
        
        goal_base = transform_pose_to_base_link(tf[0], tf[1], goal_pose)
        
        theta = math.atan2(linear_velocity[1], linear_velocity[0])
    
        # Update the direction based on angular velocity
        theta_new = theta + angular_velocity 
    
        # Normalize theta_new to the range [-pi, pi]
        theta_new = math.atan2(math.sin(theta_new), math.cos(theta_new))
        # if math.atan2(goal_base[1],goal_base[0]) >0.3:
        if i >180 and i<600:
            print(i)
            print(current_pose, '\n', goal_pose, linear_velocity, angular_velocity, tf)
            print(goal_base)
            print(math.atan2(goal_base[1],goal_base[0]))
            print(theta)
            print(theta_new)
            print()
            
        if i >600:
            break
            




181
(5.745304161781302, 8.050700985499304, 0.7311900649248584) 
 (5.37913466972676, 8.984814881587308, 0.9039268491624423) [ 7.01058622e-03 -2.42848031e-05] 0.005761557662102388 ([5.745304161781302, 8.050700985499304, 0.0], [0.0, 0.0, 0.7311900649248584, 0.6821737967374453], 1724063012746450419)
[0.95723702 0.30057636 0.90392685]
0.30425457254112337
-0.003464005040585501
0.002297552621516887

182
(5.744961511411227, 8.055675006350345, 0.7326134642518882) 
 (5.376340302811504, 8.988629221735778, 0.9039803669182616) [ 9.57227928e-03 -2.37660466e-05] 0.007835424570532101 ([5.744961511411227, 8.055675006350345, 0.0], [0.0, 0.0, 0.7326134642518882, 0.6806449235811923], 1724063012811510799)
[0.95750793 0.29910486 0.90398037]
0.30277414584764867
-0.002482794017995462
0.005352630552536639

183
(5.744619590444746, 8.060409984140174, 0.7339587860485031) 
 (5.373215918656737, 8.992862783682552, 0.9040651769468733) [ 7.69751963e-03 -6.52193396e-06] 0.00629468242944573 ([5.744619590444746, 8.060409

In [1]:
! pip install transform3d